In [ ]:
import pickle
import os
import urllib.request
import tarfile
import ssl
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

ssl._create_default_https_context = ssl._create_unverified_context

# URL of the dataset
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
save_path = "cifar-10-python.tar.gz"

# Download the file
print("Downloading CIFAR-10 dataset (ignoring SSL)...")
urllib.request.urlretrieve(url, save_path)
print(f"Dataset successfully downloaded and saved as {save_path}")

data_dir = "D:\Current Emerging Trends\cifar10"

# Function to unpickle CIFAR-10 files
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding = 'bytes')
    return data_dict

# Loading CIFAR-10 dataset from local files
def load_cifar10_data(data_dir):
    X_train = []
    y_train = []

# CIFAR_10 is set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3 # This is the number of channels (RGB)
IMG_ROWS = 32 # This is the image height
IMG_COLS = 32 # This is the image width
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10 # This is the number of output classes (CIFAR-10)
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()
NUM_TO_AUGMENT = 5

# Loading CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Data Augmentation
print("Augmenting data...")
datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

# Augmenting a subset of the dataset
xtas, ytas = [], []

for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # selecting one image
    x = x.reshape((1,) + x.shape) #reshape to (1, 32, 32, 3)
    
    for x_aug in datagen.flow(x, batch_size = 1, save_to_dir = None):
        if num_aug >= NUM_TO_AUGMENT:
            break
        xtas.append(x_aug[0])
        ytas.append(y_train[i])
        num_aug += 1
        
# Converting augmented data to numpy arrays and append to original training set
xtas = np.array(xtas)
ytas = np.array(ytas)
X_train = np.concatenate((X_train, xtas), axis = 0)
y_train = np.concatenate((y_train, ytas), axis = 0)

# normalizing the dataset
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# The Model architecture
model = Sequential()

# The First Convolutional Block
model.add(Conv2D(32, (3, 3), padding = 'same', input_shape = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

# The Second Convolutional Block
model.add(Conv2D (64, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

# The summary of the model
model.summary()

# Fit the dataget
datagen.fit(X_train)

# This is the old model
model.compile(loss = 'categorical_crossentropy', optimizer = OPTIM,  metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = NB_EPOCH, validation_split = VALIDATION_SPLIT, verbose = VERBOSE)
score = model.evalaute(X_test, Y_test, batch_size = BATCH_SIZE, verbose = VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])
print("Model compiled successfully")

# The new tarin
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size = BATCH_SIZE), samples_per_epoch = X_train.shape[0], epochs = NB_EPOCH, verbose = VERBOSE)
score = model.evaluate(X_test, Y_test, batch_size = BATCH_SIZE, verbose = VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

# Saving the model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
# Saving the weights
model.save_weights('cifar10_weights.h5', overwrite = True)

# plot trainnig history
plt.plot(history.history['accuracy'], label = 'Train Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc = 'upper left')
plt.show()

plt.plot(history.history['loss'], label = 'Train Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc = 'upper left')
plt.show()

Dataset successfully downloaded and saved as cifar-10-python.tar.gz
Augmenting data...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________